In [136]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

In [137]:
dataset = pd.read_csv('/content/drive/My Drive/toy/Churn_Modelling.csv')

In [138]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [139]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [140]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [141]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [142]:
# Normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [144]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [145]:
y = torch.from_numpy(y_train)
y.shape

torch.Size([8000])

In [146]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12, 512)           # obviamente tem que bater com as features, e a input
        self.fc2 = nn.Linear(512, 512)          # 512 escolhemos aleatoriamente, sao as hidden features
        self.fc3 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
model = Net()
model

Net(
  (fc1): Linear(in_features=12, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [147]:
[param.shape for param in model.parameters()]

[torch.Size([512, 12]),
 torch.Size([512]),
 torch.Size([512, 512]),
 torch.Size([512]),
 torch.Size([2, 512]),
 torch.Size([2])]

In [148]:
for name, param in model.named_parameters():
  print(name, param.shape)

fc1.weight torch.Size([512, 12])
fc1.bias torch.Size([512])
fc2.weight torch.Size([512, 512])
fc2.bias torch.Size([512])
fc3.weight torch.Size([2, 512])
fc3.bias torch.Size([2])


In [149]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


batch_size = 64
n_epochs = 200
batch_no = len(X_train) // batch_size

train_loss = 0
train_loss_min = np.Inf
for epoch in range(n_epochs):
    for i in range(batch_no):
        start = i*batch_size
        end = start+batch_size
        x_var = Variable(torch.FloatTensor(X_train[start:end]))
        y_var = Variable(torch.LongTensor(y_train[start:end])) 
        
        optimizer.zero_grad()
        output = model(x_var)
        loss = criterion(output,y_var)
        loss.backward()
        optimizer.step()
        
        values, labels = torch.max(output, 1)
        num_right = np.sum(labels.data.numpy() == y_train[start:end])
        train_loss += loss.item()*batch_size
    
    train_loss = train_loss / len(X_train)
    if train_loss <= train_loss_min:
        print("Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(train_loss_min,train_loss))
        torch.save(model.state_dict(), "model.pt")
        train_loss_min = train_loss
    
    if epoch % 200 == 0:
        print('')
        print("Epoch: {} \tTrain Loss: {} \tTrain Accuracy: {}".format(epoch+1, train_loss,num_right / len(y_train[start:end]) ))
print('Training Ended! ')

Validation loss decreased (   inf ===> 0.504124). Saving the model...

Epoch: 1 	Train Loss: 0.5041242527961731 	Train Accuracy: 0.796875
Validation loss decreased (0.504124 ===> 0.455243). Saving the model...
Validation loss decreased (0.455243 ===> 0.438207). Saving the model...
Validation loss decreased (0.438207 ===> 0.432425). Saving the model...
Validation loss decreased (0.432425 ===> 0.426996). Saving the model...
Validation loss decreased (0.426996 ===> 0.421693). Saving the model...
Validation loss decreased (0.421693 ===> 0.418801). Saving the model...
Validation loss decreased (0.418801 ===> 0.415179). Saving the model...
Validation loss decreased (0.415179 ===> 0.412803). Saving the model...
Validation loss decreased (0.412803 ===> 0.408648). Saving the model...
Validation loss decreased (0.408648 ===> 0.404918). Saving the model...
Validation loss decreased (0.404918 ===> 0.401059). Saving the model...
Validation loss decreased (0.401059 ===> 0.396426). Saving the model..

In [150]:
X_test_var = Variable(torch.FloatTensor(X_test), requires_grad=False) 
with torch.no_grad():
    test_result = model(X_test_var)
values, labels = torch.max(test_result, 1)

In [151]:
test_result

tensor([[ 0.3625, -0.3330],
        [ 0.5312, -0.6596],
        [ 0.8603, -1.0998],
        ...,
        [ 1.2264, -1.4105],
        [ 0.9188, -1.0298],
        [ 1.1605, -1.2481]])

In [152]:
torch.max(test_result, 1)

torch.return_types.max(values=tensor([0.3625, 0.5312, 0.8603,  ..., 1.2264, 0.9188, 1.1605]), indices=tensor([0, 0, 0,  ..., 0, 0, 0]))

In [153]:
values

tensor([0.3625, 0.5312, 0.8603,  ..., 1.2264, 0.9188, 1.1605])

In [154]:
labels

tensor([0, 0, 0,  ..., 0, 0, 0])

In [155]:
from sklearn import metrics
metrics.accuracy_score(y_test, labels)

0.8585

In [156]:
x_var = Variable(torch.FloatTensor(X_train[start:end]))
x_var.dtype

torch.float32